In [1]:
# Association tests between MYC family gene amplification and ecDNA.

In [19]:
import pandas as pd

import os
import sys
sys.path.append(os.path.abspath('/mnt/c/Users/ochapman/Documents/Mesirov/Software/oscutils'))
import get
import scipy
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

In [20]:
def get_table():
    patients = get.medullo_patients()
    patients['ecDNA'] = patients['ecDNA'].fillna('no')
    patients['myc_amp'] = patients.index.isin(get_MYC_amp_patients())
    patients['mycn_amp'] = patients.index.isin(get_MYCN_amp_patients())
    return patients

def tabulate(by):
    '''
    by: iterable of columns to form multiindex.
    '''
    patients[['count']]=1
    patients = patients.groupby(['Subgroup','ecDNA']).count()

    return patients[['count']]

def map_cbtn(key=None,value=None):
    df = get.medullo_samples()
    df=df[df.index.map(lambda x: str(x).startswith('BS'))]
    if key == None or value == None:
        return df
    else:
        df["Sample_ID"] = df.index
        return df.set_index(key)[value].to_dict()
    
def preprocess_ac(df):
    # drop ampliconx suffix
    df['sample_name']=df['sample_name'].map(lambda x: '_'.join(x.split('_')[:-1]))
    # drop feature suffix
    df['feature']=df['feature'].map(lambda x: x.split('_')[0])
    # map cbtn sample names
    d=map_cbtn('Aliases','Sample_ID')
    df['sample_name']=df['sample_name'].map(lambda x: d[x] if x in d.keys() else x)
    df['sample_name']=df['sample_name'].map(lambda x: x.split('.')[0])
    # map grch37 ids
    df['sample_name']=df['sample_name'].map(lambda x: x[:-3] if x.endswith('_AA') else x)
    # map patient ids
    d=get.medullo_samples()['Patient_ID'].to_dict()
    df['Patient_ID']=df['sample_name'].map(lambda x: d[x] if x in d.keys() else x)
    return df

def get_MYC_amp_patients():
    DIR="/home/ochapman/Documents/Mesirov/medullo_ecDNA/data/AmpliconClassifier/latest"
    hg38_set=pd.read_csv(DIR+"/hg38_gene_list.tsv",sep='\t')
    hg38_set=preprocess_ac(hg38_set)
    hg38_set=hg38_set[hg38_set.gene=='MYC']
    pts = set(hg38_set['Patient_ID'])
    hg19_set=pd.read_csv(DIR+'/grch37_gene_list.tsv',sep='\t')
    hg19_set=preprocess_ac(hg19_set)
    hg19_set=hg19_set[hg19_set.gene=='MYC']
    pts = pts | set(hg19_set['Patient_ID']) | set(['MB106','MB260'])
    return pts

def get_MYCN_amp_patients():
    DIR="/home/ochapman/Documents/Mesirov/medullo_ecDNA/data/AmpliconClassifier/latest"
    hg38_set=pd.read_csv(DIR+"/hg38_gene_list.tsv",sep='\t')
    hg38_set=preprocess_ac(hg38_set)
    hg38_set=hg38_set[hg38_set.gene=='MYCN']
    pts = set(hg38_set['Patient_ID'])
    hg19_set=pd.read_csv(DIR+'/grch37_gene_list.tsv',sep='\t')
    hg19_set=preprocess_ac(hg19_set)
    hg19_set=hg19_set[hg19_set.gene=='MYCN']
    pts=pts | set(hg19_set['Patient_ID'])
    return pts

In [28]:
patients = get_table()

In [30]:
def fisherexact(tbl):
    _,p=scipy.stats.fisher_exact(tbl)
    print("p =",min(p/2,1), '(Fisher exact test)')
    t = tbl.to_numpy().flatten()
    print("Precision =",round(t[3]/(t[3]+t[1]),3))
    print("Recall =",round(t[3]/(t[3]+t[2]),3))
    return tbl
def chisq(tbl):
    chi2,p,_,_ = scipy.stats.chi2_contingency(tbl)
    print("chi^2 =",round(chi2,3),", p =",p, '(Chi-squared test)')
    t = tbl.to_numpy().flatten()
    print("Precision =",round(t[3]/(t[3]+t[1]),3))
    print("Recall =",round(t[3]/(t[3]+t[2]),3))
    return tbl

In [31]:
fisherexact(pd.crosstab(patients.ecDNA,patients.mycn_amp))

p = 2.3886277850009638e-18 (Fisher exact test)
Precision = 0.958
Recall = 0.28


mycn_amp,False,True
ecDNA,,
no,385,1
yes,59,23


In [32]:
chisq(pd.crosstab(patients.ecDNA,patients.myc_amp))

chi^2 = 29.248 , p = 6.36715125430278e-08 (Chi-squared test)
Precision = 0.65
Recall = 0.159


myc_amp,False,True
ecDNA,,
no,379,7
yes,69,13
